In [100]:
import pandas as pd 

df = pd.read_csv('data/lines.csv')
df.head()


,id,line
0,0,Те кто не нашёл счастья
1,1,Тем кто и не искал его
2,2,Тевтон как среди ястреб я
3,3,Те кто не кискал и вот
4,4,Тэйп дом и о том госпел


In [101]:
df.line = df.line.astype(str).apply(lambda x: x.split())
df.head()

,id,line
0,0,"[Те, кто, не, нашёл, счастья]"
1,1,"[Тем, кто, и, не, искал, его]"
2,2,"[Тевтон, как, среди, ястреб, я]"
3,3,"[Те, кто, не, кискал, и, вот]"
4,4,"[Тэйп, дом, и, о, том, госпел]"


In [103]:
x_train = [line[:-1] for line in df.line]
y_train = [line[1:] for line in df.line]

In [104]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.line)
x_train = tokenizer.texts_to_sequences(x_train)
y_train = tokenizer.texts_to_sequences(y_train)
print(x_train)

[[210, 35, 4, 2355], [241, 35, 2, 4, 1361], [10464, 6, 435, 10465], [210, 35, 4, 10466, 2], [10467, 152, 2, 75, 269], [10468, 2, 10469], [2, 1362, 10470, 107], [10471, 10472], [10473, 80, 19, 10474, 10475, 10476, 5], [10477, 16, 10478, 2358, 474, 2359, 10479, 1363, 6460, 5], [2, 2359, 66, 10480], [95, 474, 4651, 10481, 1016], [6463, 599, 4652, 577, 46, 10482], [368, 6, 4653, 3, 10483], [6465, 10484, 923], [1118, 553], [2033, 415], [1118, 553], [114, 115, 600, 114, 115, 72], [1118, 553], [2033, 415], [1118, 553], [114, 115, 600], [697, 35, 10487], [33, 6466, 6467], [5, 1554], [10489, 1555], [2034, 1556, 3, 3491], [320], [195, 5, 10492, 6], [475, 9, 14, 4], [1118, 553], [2033, 415], [1118, 553], [114, 115, 600, 114, 115, 72], [861, 1, 140, 476, 1, 4656, 1, 4656, 9, 4, 254, 11, 6469, 10, 601, 10493, 1, 6470, 126, 26], [382, 10494, 2361, 2362, 10495, 10496, 4657, 3493, 2904], [1, 4658, 698, 10497, 4659], [21, 1, 4, 3495, 10498, 24, 4660, 97], [13, 7, 121, 10499, 290, 1230, 5], [924, 6471, 

In [105]:
lengths = []

for sequence in x_train:
    lengths.append(len(sequence))
    
lengths = pd.Series(lengths)
lengths.describe()


count    20056.000000
mean         5.543927
std          3.365030
min          0.000000
25%          4.000000
50%          5.000000
75%          7.000000
max        146.000000
dtype: float64

In [113]:
# максимальное колическтво символов
maxlen=100

x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
y_train = pad_sequences(y_train, maxlen=maxlen, padding='post', truncating='post')
print(x_train)

[[  210    35     4 ...     0     0     0]
 [  241    35     2 ...     0     0     0]
 [10464     6   435 ...     0     0     0]
 ...
 [    1     4   179 ...     0     0     0]
 [    1   179     0 ...     0     0     0]
 [    1     4   179 ...     0     0     0]]


In [114]:
word2idx = tokenizer.word_index
idx2word = {value: key for key, value in word2idx.items()}

In [115]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Input, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [117]:
maxlen = 1024
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
vocab_size

27953

In [118]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True))
model.add(GRU(units=1024, return_sequences=True))
model.add(Dense(vocab_size))

In [119]:
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
history = model.fit(x_train, y_train, epochs=50, verbose=1)

Epoch 1/50
195/627 [========>.....................] - ETA: 33:07 - loss: 0.4930

In [ ]:
model.save("zamay.h5")
# model = load_model("zamay.h5")